# Paris Metro Shortest Tour Meta-Heursitic Optimization

In this notebook I present two kinds of algorithms to find the shortest tour of all metro stations within Paris. The requirement is that at least one pass is made through each station within Paris city limits but they may be visited more than once. Furthermore, stations outside Paris are not out of bounds(ie one may use them if desired).

The two algorithms are:

* **ant_explorer**:
Ant colony Optimization inspired algorithm where edge selection is determined by connected edges' pheromone levels and path quality. The factors used to compute path quality are the number of previous visits to corresponding neighboring nodes and edge cost.

* **smart_explorer**:
Edge selection at each node is based on the cost or time associated to the connected edges & the number of passes previously made through the stations to which those edges connect.

*Conclusion: smart_explorer outperforms ant_explorer by a significant margin*

**Shortest Time**: The best time I was able achieve with the smart explorer was **7.53527777778 hrs** (although the final run produced a slightly different result as shown in the final cell)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import string
import random

#### Please insert the correct path for the file *metro_complet.txt*

In [2]:
#file name to be loaded
#Please insert relevant file path for metro_complet.txt

filename = '/home/ec2-user/Notebooks/ParisMetro/data/metro_complet.txt'

## Some Notes:

* [smart_explorer]  -> tag to mark a cell belonging to the smart_explorer algorithm

* [ant_explorer]    -> tag to mark a cell belonging to the ant_explorer algorithm

* The cell execution time for 'heavy' cells provided are for AWS EC2 't2.2xlarge' [8 vCPUs, 32 GiB Memory]
  (Note: this isn't very different from running on iCore 7 with 8 GB RAM since no use has been made of multi-threading or multi-processing)

* Notebook includes results

## Functions

All the following function cells need to be executed.
Descriptions are provided in the comments.

#### Stations(vertices or nodes) dataframe

In [3]:
#Function: constructs vertices(nodes) dataframe from file

#Params:
#          f -> file name

def get_vertexDF(f):
    vertexID = []
    vertexName = []
    with open(f,'r',encoding = "ISO-8859-1") as fh:
         for curline in fh:
            if curline.startswith('[Vertices]'):
                continue
            elif curline.startswith('[Edges]'):
                break
            else:
                vertexID.append(int(curline[1:4]))
                vertexName.append(curline[5:-1])

    vertexDF = pd.DataFrame({'ID':vertexID,'Name':vertexName})
    return vertexDF

#### Paths(edges) dataframe

In [4]:
#Function: constructs edges dataframe from file

#Params:
#          f -> file name

def get_edgeDF(f):
    edgeS = []
    edgeD = []
    edgeC = []
    with open(f,'r',encoding = "ISO-8859-1") as fh:
         for curline in fh:
            if not curline.startswith('[Edges]'):
                continue
            else:
                break
         for curline in fh:
            elems =curline.split(" ")
            edgeS.append(int(elems[0]))
            edgeD.append(int(elems[1]))
            edgeC.append(int(float(elems[2][:-1])))

    edgeDF = pd.DataFrame({'Source':edgeS, 'Destination':edgeD, 'Cost':edgeC})
    return edgeDF

#### Total number of stations

In [5]:
#Function: returns total number of nodes(or stations)

#Params:
#          df -> vertex(node) dataframe (or the cost matrix)

def get_totalNodes(df):
    return len(df)

#### Cost(time) matrix construction

In [6]:
#Function: constructs cost matrix (sparse)

#Params:
#          dfV -> vertex(node) dataframe ; dfE -> edge dataframe

def get_costMatrix(dfV, dfE):
    rank = get_totalNodes(dfV)
    cost_matrix = np.full((rank,rank),np.inf)

    for i in range(rank):
        row = []
        for j in range(rank):
            try:
                cost_matrix[i,j] = dfE[(dfE['Source']==dfV.iloc[i,0]) & (dfE['Destination']==dfV.iloc[j,0])]['Cost'].values[0]
            except IndexError:
                continue
    return cost_matrix

#### Station aliases

In [7]:
#Function: returns a list of lists of station IDs corresponding to the same name

#Params:
#          dfV -> vertex(node) data frame

def get_sameNodes(dfV):
    
    same_list = []
    
    for i in range(0,len(dfV)):
        v_list = []
        v_list.append(i)
        for num in range(0, len(dfV)):
            if i!=num:
                if dfV.iloc[i]['Name']==dfV.iloc[num]['Name']:
                    v_list.append(num)
        same_list.append(v_list)
    
    return same_list

#### Neghboring stations

In [8]:
#Function: returns a list of lists of neigboring nodes for the current node

#Params:
#          dfE -> edge data frame

def get_neighbors(dfE,l):
    
    edge_list = []
    
    for i in range(0,l):
        e_list = []
        for num in range(0, len(dfE)):
            if i==dfE.iloc[num]['Source']:
                    e_list.append(dfE.iloc[num]['Destination'])
        edge_list.append(e_list)
    
    return edge_list

#### Edge selection functions

In [9]:
#Function: returns unvisited node with shortest path from current node

#Params:
#          c_node -> current node ; cm -> cost_matrix ; h -> current path history

#note:     if all notes are visited then returned value is np.inf since the cost matrix is initialized..
#          to np.inf for non-existing edges

def get_shortest_unvisited(c_node, cm, h):

    k=0
    tmp = np.argpartition(cm[c_node], k)[k]

    while ((tmp in h) & (cm[c_node,tmp]!=np.inf)):
        k+=1
        tmp = np.argpartition(cm[c_node], k)[k]
        
    return tmp

In [10]:
#[smart_explorer]

#Function: returns next node
#          if, unvisited node(s) exists, it returns shortest
#          else, next best unvisited node has np.inf as cost, thus returns random least visited shortest path node

#Params:
#          c_node -> current node ; cm -> cost matrix ; tmp -> next best node from function 'get_shortest_unvisited' ;
#          v -> list of visited nodes for current tour ; el -> neighboring nodes for current node


def get_nextNode(c_node, cm, tmp, v, el):
    
    choice_list = []
    
    if cm[c_node,tmp] == np.inf:
        
        opts = el[c_node]
        choice = opts[0]
        freq=v[choice]
        
        for c in opts:
            if v[c]<v[choice]:
                choice = c
                freq = v[c]
                        
        for c in opts:
            if v[c]==freq:
                choice_list.append(c)
                
        choice = random.choice(choice_list)
        t = choice

    else:
        t = tmp
    
    return t

In [11]:
#[ant explorer]

#Function: fetches the next path
#          If path(s) connecting to unvisited node(s) exist , it returns the shortest one.
#          If not then the selected path depends on the computed probabilites assigned to each connecting edge.
#          This probability is determined by edge pheromone level(influence of which is controlled by constant alpha) and..
#          the quality of the path(influence of which is controlled by constant alpha).
#          The path quality here is determined by 1/(path distance*exp(#of previous visits of all ants to node at the end of this path))

#Params:   c_node -> current node ; c_mat -> cost matrix ; tmp -> next best node from function 'get_shortest_unvisited' ;
#          ph_mat -> pheromone matrix ; v -> list of visited nodes for current tour ;
#          el -> neighboring nodes for current node ; alpha -> pheromone influence constant ;
#          beta -> path quality influence constant

def get_nextPath(c_node, c_mat, tmp, ph_mat, v, el, alpha, beta):
    

    
    if c_mat[c_node,tmp] == np.inf:
        
        opts = el[c_node]

        
        prob = []
        sum_influence = 0

        for c in opts:
            sum_influence += ph_mat[c_node, c]**alpha * (1/(c_mat[c_node, c]*np.exp(v[c])))**beta

        for c in opts:
            prob.append(((ph_mat[c_node, c]**alpha * (1/(c_mat[c_node, c]*np.exp(v[c])))**beta)) / sum_influence)

        t = opts[prob.index(max(prob))]

    else:
        t = tmp
    
    return t
    #path selection probability for possible paths v of current node

#### Pheromone update

In [12]:
#[ant explorer]

#Function: updates the pheromones deposited on all edges that make up a tour

#Params:
#          tour -> current solution/tour ; ph_mat -> pehromone matrix ; Q -> cost function dependent constant ;
#          F -> total cost of a tour ; rho -> pheromone evaporation rate

def update_pheromones(tour,ph_mat,Q,F,rho):
    
    #pheromone amount deposited by current ant on each edge
    delta_tao = Q/F
    
    
    #pheromone update for each edge of current solution
    for i in range(0,len(tour)-1):
        ph_mat[tour[i],tour[i+1]] = (1-rho)*ph_mat[tour[i],tour[i+1]] + delta_tao
    return ph_mat

#### Nodes within Paris
This includes all stations with different IDs but same name. Same name nodes will be removed in the algorithm. 

In [30]:
#Function: returns a list of the stations within Paris city limits (ie bounded by 'Porte' stations)

#Params:
#          cm -> cost matrix ; sl -> same nodes list ; el -> neighboring nodes for current node ;
#          dfV -> vertex(nodes) dataframe ; pt -> node of interest 'Jean Jaures' station to be included(value = 'w')...
#          or exluded from among the city nodes with value w instructing an inclusion

def city_nodes(cm, sl, el, dfV, pt):
    
    l = get_totalNodes(cm)
    start_list = [r for r in range(0,l)]
    unvisited = set(range(l))
    pattern = 'Porte'
    
    #option to consider station 'Boulogne, Jean Jaurès' as part of inner city stations
    node_of_interest = 36
    
    while len(unvisited)>100:
        
        start = random.choice(start_list)

        s = start
        hist = [s]

        visited = np.zeros(shape=(l))
        visited[s] += 1
        unvisited = set(range(l))

        for v in range(len(sl[s])):
            unvisited.remove(sl[s][v])
        
        repeats = 0
        while (repeats < 3000):

            tmp = get_shortest_unvisited(s, cm, hist)
            t = get_nextNode(s, cm, tmp, visited, el)

            for v in range(len(sl[t])):
                    try:
                        unvisited.remove(sl[t][v])
                    except KeyError:
                        repeats += 1
                        continue

            hist.append(t)
            visited[t] += 1
            if pattern in dfV.iloc[t]['Name']:
                continue
            else:
                s=t
    
    if pt == 'w':
        hist.append(36)
    return set(hist), hist

#### Solver output

In [14]:
#Function: display and returns results of the solver or algorithm
#Params:
#          c_r -> costs for the tours ; h_l -> list of feasible tour paths ; 
#          uv_l -> unvisited list(if any) for each solution (unvisited refers to nodes that are required to be..
#          to be visited and thus must be empty)

def solver_results(c_r, h_l, uv_l):

    best_index = np.argmin(c_r)
    
    best_init = h_l[best_index][0]
    least_cost = c_r[best_index]
    least_cost_hrs = c_r[best_index]/(60*60)
    unvisited_set = uv_l[best_index]
    tour_length = len(h_l[best_index])
    nos_stations = len(set(h_l[best_index]))

    print('Best Starting station', best_init)
    print('Total Time in seconds:', least_cost)
    print('Total Time in hrs:', least_cost_hrs)
    print('Un-visited Set',unvisited_set)
    print('Tour length(# of stations)', tour_length)
    print('# of unique stations', nos_stations)
    
    return best_init, tour_length, nos_stations, least_cost_hrs

## Ant Explorer Algorithm

#### Edge Selection

The $kth$ ant moves from node $x$ to node $y$ with probability :
\begin{equation*}
p_{xy}^k = \frac{(\tau_{xy}^\alpha)(\eta_{xy}^\beta)}{\sum_{z\in_{allowed_x}}(\tau_{xy}^\alpha)(\eta_{xy}^\beta)}
\end{equation*}

where $\eta_{xy}$ gives the path quality, with $V_y$ being the number of previous visits of node $y$ connected to current node $x$ in the current partial tour,

\begin{equation*}
\eta_{xy} = \frac{1}{d_{xy}e^{V_y}}
\end{equation*}


### Pheromone Update

When $kth$ ant  finishes a tour all edges are updated as follows, where $\tau_{xy}$ being the pheromone level of edge belonging to node $x$ and $y$,

\begin{equation*}
\tau_{xy} \leftarrow (1 - \rho)\tau_{xy} + \sum_k\Delta\tau_{xy}^k
\end{equation*}

where $\Delta\tau_{xy}^k$ is given by,

\begin{equation*}
\Delta\tau_{xy}^k = 
\begin{cases}
 \frac{Q}{L_k} & \text{if ant $k$ uses path $xy$ in its tour} \\
 0 & \text{otherwise}
\end{cases}
\end{equation*}

here $Q$ is some problem dependent constant and $L_k$ is the cost of $kth$ ant's tour

In [16]:
#[ant_explorer]

#ant_explorer algorithm for shortest tour optimization - Ant Colony inspired

#path selection strategy priority: (1)shortest unvisited ~ (2)best visited(combination of pheromone level & path quality)

#Starting node: random

#Function: returns list of costs for all solutions,
#                  list of tour paths of all solutions,
#                  list of any unvisited nodes(purpose being to confirm that all required nodes were visited)

#Params:
#          t_b -> time budget in seconds ; itr -> # of iterations per node ; cm -> cost matrix ; 
#          sl -> same nodes list ; el -> neighboring nodes for current node ; dfV -> vertex(nodes) dataframe ; 
#          pt -> node of interest 'Jean Jaures' station (for more info see function 'city_nodes') ;
#          alpha -> pheromone influence constant ; beta -> path quality influence constant
#          Q -> cost function dependent constant ; rho -> pheromone evaporation rate


def ant_explorer(t_b, itr, cm, sl, el, dfV, pt, alpha, beta, Q, rho):

    city_n, dummy = city_nodes(cm, sl, el, dfV, pt)
    count = 0
    l = get_totalNodes(cm)
    hist_list = []
    unvisited_list = []
    cost_runs = np.zeros(shape=itr)
    ph_mat = np.ones(shape=(l,l))
    
    all_cities = [c for c in range(0,l)]
        
    for y in range(0,itr):
        
        i = random.choice(all_cities)
        s = i
        hist = [s]
        costs = 0

        visited = np.zeros(shape=(l))
        visited[s] += 1
        unvisited = city_n.copy()

        for v in range(len(sl[s])):
            try:
                unvisited.remove(sl[s][v])
            except KeyError:
                continue

        while (len(unvisited)>0) & (costs<t_b):
            
            tmp = get_shortest_unvisited(s, cm, hist)
            t = get_nextPath(s, cm, tmp, ph_mat, visited, el, alpha, beta)
            
            for v in range(len(sl[t])):
                    try:
                        unvisited.remove(sl[t][v])
                    except KeyError:
                        continue

            hist.append(t)
            costs+=cm[s,t]
            s = t
            visited[s] += 1

        cost_runs[count] = costs
        hist_list.append(hist)
        ph_mat = update_pheromones(hist,ph_mat,Q,costs,rho)
        unvisited_list.append(unvisited)
        count+=1
    
    return cost_runs, hist_list, unvisited_list

## Smart Explorer Algorithm

In [17]:
#[smart_explorer]

#smart_explorer algorithm for shortest tour optimization

#path selection strategy priority: (1)shortest unvisited ~ (1)random least visited

#Starting node: algortihm iterates over all nodes with each node iterated over 'itr' times (due to some randomness
#               in the selection strategy; see function 'get_nextNode' for more info)

#Function: returns list of costs for all solutions,
#                  list of tour paths of all solutions,
#                  list of unvisited nodes (if any) -> purpose being to confirm that all required nodes were visited

#Params:
#          t_b -> time budget in seconds (20 hrs) ; itr -> # of iterations per node ; cm -> cost matrix ; 
#          sl -> same nodes list ; el -> neighboring nodes for current node ; dfV -> vertex(nodes) dataframe ; 
#          pt -> node of interest 'Jean Jaures' station (for more info see function 'city_nodes')


def smart_explorer(t_b, itr, cm, sl, el, dfV, pt):

    city_n, dummy = city_nodes(cm, sl, el, dfV, pt)
    count = 0
    l = get_totalNodes(cm)
    hist_list = []
    unvisited_list = []
    cost_runs = np.zeros(shape=l*itr)
    

    for i in range(0,l):
        
        for y in range(0,itr):

            s = i
            hist = [s]
            costs = 0

            visited = np.zeros(shape=(l))
            visited[i] += 1
            unvisited = city_n.copy()

            for v in range(len(sl[i])):
                try:
                    unvisited.remove(sl[i][v])
                except KeyError:
                    continue

            while (len(unvisited)>0) & (costs<t_b):

                tmp = get_shortest_unvisited(s, cm, hist)
                t = get_nextNode(s, cm, tmp, visited, el)

                for v in range(len(sl[t])):
                        try:
                            unvisited.remove(sl[t][v])
                        except KeyError:
                            continue

                hist.append(t)
                costs+=cm[s,t]
                s = t
                visited[s] += 1

            cost_runs[count] = costs
            hist_list.append(hist)
            unvisited_list.append(unvisited)
            count+=1
    
    return cost_runs, hist_list, unvisited_list

## Data Preparation
* Dataframes are created for the vertices(nodes) and edges.
* The cost matrix is constructed out of the travel times between the stations.
* One list is populated with lists of nodes corresponding to the same station.
* And another one with lists of neighboring stations for each station.

In [18]:
%%time
#cell exec time ~ 4mins 7s

#data preparation for vertices, edges, cost matrix, same nodes & neighboring nodes

dfV = get_vertexDF(filename)
dfE = get_edgeDF(filename)
l = get_totalNodes(dfV)
dist_mat = get_costMatrix(dfV, dfE)
same_list = get_sameNodes(dfV)
neighbor_list = get_neighbors(dfE,l)

CPU times: user 4min 10s, sys: 2.51 s, total: 4min 13s
Wall time: 4min 7s


## Run the algorithms

#### Ant Explorer

In [33]:
%%time
#10000 iterations:  cell exec time ~ 9mins 5s       Best tour in hrs: 8.40611111111
#30000 iterations:  cell exec time ~ 26mins 47s     Best tour in hrs: 8.34833333333

#[ant_explorer]


#evaporation rate
rho = 0.5
#pheromone influence
alpha = 0.5
#edge quality influence
beta = 1.5
#constant Q
Q = 2500

#run the algorithm
iterations = 30000
time_budget = 72000
cost_list, history_list, unvisited_list = ant_explorer(time_budget,iterations,dist_mat,same_list,neighbor_list,dfV,'w', alpha, beta, Q, rho)

#output the results
ae_start, ae_length, ae_stations, ae_cost_hrs = solver_results(cost_list, history_list, unvisited_list)

Best Starting station 149
Total Time in seconds: 30054.0
Total Time in hrs: 8.34833333333
Un-visited Set set()
Tour length(# of stations) 605
# of unique stations 370
CPU times: user 26min 48s, sys: 2.95 s, total: 26min 51s
Wall time: 26min 47s


#### Smart Explorer

In [32]:
%%time
#25 iterations:  cell exec time ~  7mins 25s      Best tour in hrs: 7.92777777778 
#50 iterations:  cell exec time ~  14mins 48s     Best tour in hrs: 7.8575
#100 iterations:  cell exec time ~ 29mins 16s     Best tour in hrs: 7.62638888889

#[smart_explorer]

#run the [smart_explorer] algorithm
iterations = 100
time_budget = 72000
cost_list, history_list, unvisited_list = smart_explorer(time_budget,iterations,dist_mat,same_list,neighbor_list,dfV,'w')

#output the results
se_start, se_length, se_stations, se_cost_hrs = solver_results(cost_list, history_list, unvisited_list)

Best Starting station 167
Total Time in seconds: 27455.0
Total Time in hrs: 7.62638888889
Un-visited Set set()
Tour length(# of stations) 533
# of unique stations 363
CPU times: user 29min 17s, sys: 3.09 s, total: 29min 20s
Wall time: 29min 16s


## Conclusion
#### Table of comparison

The selection strategies of these two algorithms only differ when all neigboring edges of a node have already been visited.

In the case of the smart_explorer, an element of randomness is only introduced when encountering visited nodes that have been previously visited the same number of times.

The ant_explorer, just as with smart_explorer, first exhausts all edges connected to unvisited nodes, and then applies the ACO inspired selection strategy but to the set of *all* visited nodes.

Since both algorithms are highly influenced by the number of previous node visits of current tour *in progress* and given the fact that the smart_explorer allows some flexibility facilitated by random selection over a much smaller search space as compared to that of a TSP problem, for which a solution could be brute forced in polynomial time, one could perhaps conclude that the smart_explorer will outperform the ant_explorer every single time (ie given enough iterations).

In [34]:
d = {'Algorithm': ['Ant Explorer', 'Smart Explorer'], 'Starting Station': [ae_start, se_start], 
     '# of stations':[ae_length, se_length], '#of unique stops':[ae_stations, se_stations],
     'Total Time':[ae_cost_hrs,se_cost_hrs]}
df = pd.DataFrame(data=d)

df[['Algorithm','Starting Station','# of stations','#of unique stops','Total Time']]

,Algorithm,Starting Station,# of stations,#of unique stops,Total Time
0,Ant Explorer,149,605,370,8.348333
1,Smart Explorer,167,533,363,7.626389


## Possible Improvements

* Ant Explorer: for each randomly selected starting node a grid search over a range pertaining to each of the hyperparameters Q, rho, alpha and beta, might yield significantly better results (essentially hyperparameter tuning for meta-heuristic algorithms). I implemented the grid search but decided against using it since it is too computationally expensive.

* Smart Explorer: it might be worth experimenting with randomness in selection strategy over the entire set of already visited neigboring edges post exhaustion of the unvisited set of edges.